## Video Game Character Project
#### Part II:
Apply what you have done in the previous project and learned in the previous lectures to update and delete data related to your video game characters within your MongoDB database.

#### Import Dependencies

In [67]:
import copy
import numpy as np

from datetime import datetime 
from pymongo import MongoClient

#### Player and Items Python Classes 

In [68]:
class Player:
    def __init__(self, name="player", max_health=50, max_energy=25, items=[], gold=0):
        self.name = name
        self.health = max_health 
        self.max_health = max_health
        self.energy = max_energy
        self.max_energy = max_energy
        self.items = copy.deepcopy(items)
        self.gold = 0
        
        
    def attack(self, player):
        energy_cost = 5
        
        if self.energy >= energy_cost:
            attack_strength = np.random.randint(7, 13)
            player.health -= attack_strength
            self.energy -= energy_cost
            print("{} attacked {} for {} damage".format(self.name, player.name, attack_strength))
        else:
            print("{} does not have enough energy to attack {}".format(self.name, player.name))
        
        
    def heal(self, amount):
        self.health += amount
        
        if self.health > self.max_health:
            self.health = self.max_health
 
    def restore_energy(self, amount):
        self.energy += amount
        
        if self.energy > self.max_energy:
            self.energy = self.max_energy

    def rest(self):
        self.health = self.max_health
        self.energy = self.max_energy
        
    def stats(self):
        return vars(self)
        
        
    def use_item(self, item_name):
        try: 
            item = next(item for item in self.items if item.name == item_name)
            item.quantity -= 1

            for effect in item.effects:
                for method, value in effect.items():
                    class_method = getattr(self, method)
                    class_method(value)

            print("{} used item: {}".format(self.name, item.name))
                    
            if item.quantity == 0:
                self.items.remove(item)         
                
        except:
            print("{} does not have any {}s".format(self.name, item_name))

In [69]:
class Item:
    def __init__(self, name, quantity, effects=[]):
        self.name = name
        self.quantity = quantity 
        self.effects = effects
        
    def __repr__(self):
        return "Item(name={}, quantity={}, effects={})".format(self.name, self.quantity, self.effects)

In [70]:
class NPC(Player):
    pass

#### Project Instructions

In [71]:
# TODO:
# 1) Initalize a MongoDB Client object to connect to your database with
client=MongoClient('localhost',27017)
db=client['video_game']


In [72]:
# TODO:
# 2) Copy your function from project 1 that takes in a Player object and inserts it into the database 
#    make sure to include the new player attribute "gold"
def convert_player_obj_to_dict(player_obj):
    print("convert_player_obj_to_dict-player_obj",player_obj)
    player_dict=copy.deepcopy(vars(player_obj))
    print("convert_player_obj_to_dict-player_dict",player_dict)
    
    items_dict_list=[]
    for item_obj in player_dict["items"]:
        item_dict=vars(item_obj)
        items_dict_list.append(item_dict)
    
    player_dict["items"]=items_dict_list
    return player_dict

def insert_player_by_obj(player_obj,check_for_duplicates=True):
    if check_for_duplicates:
        duplicate_player=db.players.find_one({"name":player_obj.name})
        if duplicate_player!=None:
            return duplicate_player["_id"]
    # make a deep copy else whenever player_obj changes player_dict will also change
    # to insert in mongodb we need to run vars and convert into dictionary
    # to insert in mongodb we need to run vars and convert into dictionary
    player_dict=convert_player_obj_to_dict(player_obj)
    return db.players.insert_one(player_dict).inserted_id


# 3) Copy your function from project 1 that is able to find a Player in the database by searching for their name
def find_player_by_name(name):
    return db.players.find_one({"name":name})


# 4) Copy your function from project 1 that loads the data from the above function and returns a Player object 
#    configured with that data, make sure to include the new player attribute "gold"
def convert_player_dict_to_obj(player_dict):
    p=player_dict
    
    items_list=[]
    for item in p["items"]:
        item_obj=Item(item["name"],item["quantity"],item["effects"])
        items_list.append(item_obj)
        
    player_obj=Player(p["name"],p["max_health"],p["max_energy"],items_list)        
    player_obj.health=p["health"]
    player_obj.energy=p["energy"]
    
    return player_obj

# 5) Create a function which returns a Player object given a name to look for in the database as an input.
#    Hint: use the functions you have in steps 3 and 4
def get_player_obj_by_name(player_name):
    player_dict = find_player_by_name(player_name)
    return convert_player_dict_to_obj(player_dict)

In [73]:
# TODO:
# 6) Create a function which takes in a NPC object and inserts it into the database.
#    Hint: You should probably organize the NPC's into a seperate MongoDB collection than the players. 
def insert_npc_by_obj(npc_obj,check_for_duplicates=True):
    if check_for_duplicates:
        duplicate_npc=db.npcs.find_one({"name":npc_obj.name})
        if duplicate_npc!=None:
            return duplicate_npc["_id"]
    # make a deep copy else whenever npc_obj changes npc_dict will also change
    # to insert in mongodb we need to run vars and convert into dictionary
    # re-use the convert_player_obj_to_dict method
    npc_dict=convert_player_obj_to_dict(npc_obj)
    return db.npcs.insert_one(npc_dict).inserted_id

# 7) Create a function that allows you to find an NPC in the database by name. 
def find_npc_by_name(name):
    return db.npcs.find_one({"name":name}) 


# 8) Create a function which given the NPC name, will return a NPC object. 
def get_npc_obj_by_name(npc_name):
    npc_dict=find_npc_by_name(npc_name)
    print("npc_dict",npc_dict)
    player_obj=convert_player_obj_to_dict(npc_dict)
    print("player_obj",player_obj)
    npc_obj=NPC()
    npc_obj.__dict__=player_obj.__dict__
    return npc_obj


In [74]:
# TODO:
# 9) Create a function which will update information in your database given the player object
def update_player_by_obj(player_obj):
    player_dict=convert_player_obj_to_dict
    return db.players.update_one({"name":player_dict["name"]},{"$set":player_dict})

# 10) Create a function which will delete a player in the database given the player's name
def delete_player_by_name(player_name):
    return db.players.delete_one({"name":player_name})

# 11) Create a function to insert a "battle log" by dict
#     Hint: You should probably organize the battle logs in a seperate MongoDB collection.
def insert_battle_log_by_dict(battle_result_dict):
    return db.battle_log.insert_one(battle_result_dict).inserted_id

#### BattleNPC Python Class

In [75]:
class BattleNPC:
    def __init__(self, player_name, npc_name, gold_reward):
        self.verify_player_and_npc_name_valid(player_name, npc_name)
        self.player = get_player_obj_by_name(player_name)
        self.npc = get_npc_obj_by_name(npc_name)
        self.gold_reward = int(gold_reward)
        self.start_time_utc = None

    def prompt_user_action(self):
        if len(self.player.items) > 0:
            acceptable_actions = ["a", "i", "q"]
            prompt = "Do you want to Attack (a), Use an Item (i), or Quit (q): "
            user_action = input(prompt)
        else:
            acceptable_actions = ["a", "q"]
            prompt = "Do you want to Attack (a) or Quit (q): "
            user_action = input(prompt)
            
        while user_action not in acceptable_actions:
            print("Invalid option, acceptable options are: ")
            print(acceptable_actions)
            user_action = input(prompt)

        return user_action


    def prompt_player_item(self, player_obj):
        for index, item in enumerate(player_obj.items):
            print("[{}] - {}".format(index, item))

        item_index = int(input("Which item do you want to use?"))
        while item_index not in range(len(player_obj.items)):
            print("Invalid item index.")
            item_index = int(input("Which item do you want to use?"))
            
        return player_obj.items[item_index].name


    def execute_player_action(self, user_action):
        if user_action == "a":
            self.player.attack(self.npc)
        elif user_action == "i":
            item_name = self.prompt_player_item(self.player)
            self.player.use_item(item_name)

    
    def npc_action(self):
        if len(self.npc.items) > 0:
            npc_action = np.random.choice(["a", "i"], 1, [0.7, 0.3])[0]
        else:
            npc_action = "a"
            
        if npc_action == "i":
            item_index = np.random.randint(0, len(self.npc.items))
            self.npc.use_item(self.npc.items[item_index].name)
        else:
            self.npc.attack(self.player)
        
    
    def generate_battle_log_dict(self, result):
        battle_log_dict = {
            "start_time_utc": self.start_time_utc,
            "player_name": self.player.name,
            "npc_name": self.npc.name,
            "result": result
        }
        return battle_log_dict

    def verify_player_and_npc_name_valid(self, player_name, npc_name):
        assert find_player_by_name(player_name) != None, \
                "Player with name '{}' does not exist in database".format(player_name)
        
        assert find_npc_by_name(npc_name) != None, \
                "NPC with name '{}' does not exist in database".format(npc_name)
            
    
    def start(self):
        self.start_time_utc = datetime.utcnow()
        
        round_num = 0
        while self.player.health > 0 and self.npc.health > 0:
            print("\n-- Round {}. Player(health={}, energy={}), NPC(health={}, energy={})".format(
                  round_num, self.player.health, self.player.energy, self.npc.health, self.npc.energy))
            
            user_action = self.prompt_user_action()

            if user_action == "q":
                print("Quitting battle...")
                battle_result = "Player {} quit against {}".format(self.player.name, self.npc.name)
                insert_battle_log_by_dict(self.generate_battle_log_dict(battle_result))
                return None
            else:
                self.execute_player_action(user_action)

            if self.npc.health > 0:
                self.npc_action()  
                
            update_player_by_obj(self.player)    
            round_num += 1

        if self.player.health > 0 and self.npc.health <= 0:
            self.player.gold += self.gold_reward 
            battle_result = "Player {} won the battle against {}".format(self.player.name, self.npc.name) 
        else:
            battle_result = "Player {} was beaten by {}".format(self.player.name, self.npc.name)
          
        print("\n-- " + battle_result + " --")
        self.player.rest()
        update_player_by_obj(self.player)
        insert_battle_log_by_dict(self.generate_battle_log_dict(battle_result))

#### Use the functions and classes above to create a Player and an NPC, and start a battle between them

In [76]:
# TODO:
# 12) Create a Player and NPC, optionally give them items
npc_items=[Item("health_potion",2,[{"heal":10}])]
# npc=NPC("Goblin",50,25,npc_items)
npc=NPC("Cyclops",50,25,npc_items)

player_items=[
    Item("greater_health_potion",2,[{"heal":25}]),
    Item("greater_energy_potion",2,[{"restore_energy":25}]),
]

player=Player("Kratos",60,35,player_items)

player.stats()

{'name': 'Kratos',
 'health': 60,
 'max_health': 60,
 'energy': 35,
 'max_energy': 35,
 'items': [Item(name=greater_health_potion, quantity=2, effects=[{'heal': 25}]),
  Item(name=greater_energy_potion, quantity=2, effects=[{'restore_energy': 25}])],
 'gold': 0}

In [77]:
# 13) Insert Player into MongoDB
insert_player_by_obj(player)

ObjectId('644f698339c72898312cbbae')

In [78]:
# 14) Insert NPC into MongoDB
insert_npc_by_obj(npc)

ObjectId('644f698439c72898312cbbaf')

In [79]:
# 15) Start a battle between the Player and NPC you created using the BattleNPC class
#     Hint: After initalizing BattleNPC, you will need to call its start() method
battle=BattleNPC("Kratos","Cyclops",500)
battle.start()

npc_dict {'_id': ObjectId('644f698439c72898312cbbaf'), 'name': 'Cyclops', 'health': 50, 'max_health': 50, 'energy': 25, 'max_energy': 25, 'items': [{'name': 'health_potion', 'quantity': 2, 'effects': [{'heal': 10}]}], 'gold': 0}
convert_player_obj_to_dict-player_obj {'_id': ObjectId('644f698439c72898312cbbaf'), 'name': 'Cyclops', 'health': 50, 'max_health': 50, 'energy': 25, 'max_energy': 25, 'items': [{'name': 'health_potion', 'quantity': 2, 'effects': [{'heal': 10}]}], 'gold': 0}


TypeError: vars() argument must have __dict__ attribute